# ECE 498 DS Mini Project 2

## Akhilesh Somani - somani4,
## Gowtham Kuntumalla - gowtham4,
## Manan Mehta - mananm2

# Task 1: Data Cleaning and Visual Inspection

## 0. Getting Started With the Data

In [1]:
import numpy as np
import pandas as pd
import scipy as sp

In [2]:
he0_df=pd.read_csv("RelativeAbundance_HE0.csv")

### 1. In the context of statistical analysis, why do biologists need multiple samples to identify microbes with significantly altered abundance?

Biologists need muliple samples to reduce the standard deviation of the data correpsonding to each microboe. This helps them to conclude, with a greater confidence, which mircobes are present in more numbers than usual.

### 2. How many samples were analyzed?
Number of samples analyzed were: 764

In [3]:
len(he0_df.set_index("Microbe Name").columns)

764

### 3. How many microbes were identified?
Number of different micorbes idenified were: 149

In [4]:
he0_df.groupby("Microbe Name").count().sum()[0]

149

## 1. Bayesian Network for Quality Control

### a. Joint Probability Factorization
T = storage temp,
M = collection method,
C = contamination,
L = Lab time,
Q = quality

$$ P(joint) = P(Q,C,L,M,T) = P(Q|C,L,M,T) * P(C|M,T,L) * P(M|T,L) * P(T|L) * P(L)$$ 
$$ P(Q,C,L,M,T) = P(Q|C,L) * P(C|M,T) * P(M) * P(T) * P(L)$$

### b. Parameters Required
Number of values taken by:
C=2,
L=2,
T=2,
M=2,
Q=2

Parameters needed are: (2x2)+(2x2)+1+1+1 = 11

### c. Constructing Probability Tables

In [5]:
qc_df=pd.read_csv("QualityControl.csv")

In [6]:
#P(T),P(M),P(T)
feature=["strtmp","coll","labtime"]
sub_features=[["cold","cool"],["nurse","patient"],["short","long"]]
for i in range(3):
    z=qc_df.groupby(feature[i]).count()
    z.loc[sub_features[i][0]], z.loc[sub_features[i][1]] = z.loc[sub_features[i][0]]/z.sum(), z.loc[sub_features[i][1]]/z.sum()
    print(z.loc[:,'qual'])
    print("\n")

strtmp
cold    0.8982
cool    0.1018
Name: qual, dtype: float64


coll
nurse      0.8976
patient    0.1024
Name: qual, dtype: float64


labtime
long     0.2044
short    0.7956
Name: qual, dtype: float64




In [7]:
#P(Q|C,L)
z=qc_df.groupby(["cont","labtime","qual"]).count()
temp=qc_df.groupby(["cont","labtime"]).count()
for i in ['high','low']:
    for j in ['long','short']:
        for k in ['bad','good']:
            z.loc[i,j,k]=z.loc[i,j,k]/temp.loc[i,j]
        
print(z.loc[:,'strtmp'])

cont  labtime  qual
high  long     bad     0.966102
               good    0.033898
      short    bad     0.064257
               good    0.935743
low   long     bad     0.080997
               good    0.919003
      short    bad     0.042907
               good    0.957093
Name: strtmp, dtype: float64


In [8]:
#P(C|M,T)
z=qc_df.groupby(["coll","strtmp","cont"]).count()
temp=qc_df.groupby(["coll","strtmp"]).count()
for i in ['nurse','patient']:
    for j in ['cold','cool']:
        for k in ['high','low']:
            z.loc[i,j,k]=z.loc[i,j,k]/temp.loc[i,j]
        
print(z.loc[:,'qual'])

coll     strtmp  cont
nurse    cold    high    0.043983
                 low     0.956017
         cool    high    0.088435
                 low     0.911565
patient  cold    high    0.076577
                 low     0.923423
         cool    high    0.838235
                 low     0.161765
Name: qual, dtype: float64


### d. Final Probability Tables: Calculate P(Quality|Storage Temp, Collection Method, Lab Time)

In [9]:
#P(Q|T,M,L)

cpd=qc_df.groupby(["strtmp","coll","labtime","qual"]).count()
temp=qc_df.groupby(["strtmp","coll","labtime"]).count()

for i in ['cold','cool']:
    for j in ['nurse','patient']:
        for k in ['long','short']:
            for l in ['bad','good']:
                cpd.loc[i,j,k,l]=cpd.loc[i,j,k,l]/temp.loc[i,j,k]
        
print(cpd)

                                 cont
strtmp coll    labtime qual          
cold   nurse   long    bad   0.112038
                       good  0.887962
               short   bad   0.044888
                       good  0.955112
       patient long    bad   0.137931
                       good  0.862069
               short   bad   0.056022
                       good  0.943978
cool   nurse   long    bad   0.177215
                       good  0.822785
               short   bad   0.027624
                       good  0.972376
       patient long    bad   0.882353
                       good  0.117647
               short   bad   0.039216
                       good  0.960784


### e. Cleaning the dataset

In [10]:
bi_he0_df=pd.read_csv('BayesInferenceHE0.csv')
bi_he1_df=pd.read_csv('BayesInferenceHE1.csv')

bi_he0_df['prob']=0
bi_he1_df['prob']=0

bi_he0_df['prob']=[cpd.loc[bi_he0_df['strtmp'].iloc[i],bi_he0_df['coll'].iloc[i],bi_he0_df['labtime'].iloc[i],'bad'][0]\
                           for i in range(len(bi_he0_df))]

bi_he1_df['prob']=[cpd.loc[bi_he1_df['strtmp'].iloc[i],bi_he1_df['coll'].iloc[i],bi_he1_df['labtime'].iloc[i],'bad'][0]\
                           for i in range(len(bi_he1_df))]

idx1=(bi_he0_df['prob']>0.5)
print("Samples that are bad in HE0 Dataset are:\n")
print(bi_he0_df['SampleName'][idx1])

idx2=(bi_he1_df['prob']>0.5)
print("Samples that are bad in HE1 Dataset are:\n")
print(bi_he1_df['SampleName'][idx2])

Samples that are bad in HE0 Dataset are:

699    HE0Sample_699
700    HE0Sample_700
701    HE0Sample_701
702    HE0Sample_702
703    HE0Sample_703
704    HE0Sample_704
705    HE0Sample_705
706    HE0Sample_706
707    HE0Sample_707
708    HE0Sample_708
709    HE0Sample_709
710    HE0Sample_710
711    HE0Sample_711
712    HE0Sample_712
713    HE0Sample_713
714    HE0Sample_714
715    HE0Sample_715
716    HE0Sample_716
717    HE0Sample_717
718    HE0Sample_718
719    HE0Sample_719
720    HE0Sample_720
721    HE0Sample_721
722    HE0Sample_722
723    HE0Sample_723
724    HE0Sample_724
725    HE0Sample_725
726    HE0Sample_726
727    HE0Sample_727
728    HE0Sample_728
           ...      
734    HE0Sample_734
735    HE0Sample_735
736    HE0Sample_736
737    HE0Sample_737
738    HE0Sample_738
739    HE0Sample_739
740    HE0Sample_740
741    HE0Sample_741
742    HE0Sample_742
743    HE0Sample_743
744    HE0Sample_744
745    HE0Sample_745
746    HE0Sample_746
747    HE0Sample_747
748    HE0Sam

## 2. Data Standardization

### 1. Verify Relative Abundance

### 2. Discussion on Relative Abundance Data

## 3. Visual Inspection

# Task 2: Statistical Analysis

## 1. Kolmogorov-Smirnov (KS) Test

### a. p-value for Two-Sample KS Test

### b. Null Hypothesis of the KS Test in our Context

### c. Number of Genes with Significantly Altered Expression at Various Alpha Values

## 2. Multiple Testing

### a. P-value

### b. P-value Distribution

### c. Expected Number of Significant p-values

### d. Q-Q plot of p-values

#### i. 

#### ii. 

#### iii. 

#### iv. 

#### v. 

#### vi. 

### e. Follow-up Questions

#### i. 

#### ii. 

# Task 3: Dimensionality Reduction and Clustering

## 1. PCA

### a. Performing PCA


### b. Ordering Components

### c. Scatter Plot of Microbe Abundance

## 2. t-SNE

### a. Performing t-SNE

### b. Comparing with PCA

## 3. Clustering

### a. K-Means

### b. GMM

### c. Single and Complete Linkage Hierarchical

### d. Single vs Complete Link Hierarchical Clustering

### e. Comparing Clustering Techniques

### f. What do Clusters Represent

### g. Deciding on Number of Clusters

# Task 4: Interpreting Results

## 1. Identify Microbes with Altered Abundance Levels

### a. Determine if each HE1 Subpopulation shows altered abundance levels

### b. Identify Most Similar HE0 Clusters to Affected HE1 Clusters

### c. Identify Microbes with Significantly Altered Relative Abundance

## 2. Identify How Abundance Changes

### a. Microbes which increase and decrease

### b. Biological Relations